In [1]:
import requests
import os
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
def fetch_and_save_stats(url, filename_prefix, div_id, table_id):
    driver = webdriver.Chrome()
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, div_id)))


    page_source = driver.page_source
    soup = bs(page_source, 'html.parser')

    div = soup.find('div', {'id': div_id})
    table = div.find('table', {'id': table_id})

   
    # Get the first and second header rows
    headers = []
    thead = table.find('thead')
    first_row = thead.find_all('tr')[0]  
    second_row = thead.find_all('tr')[1]  

    # Combine headers
    combined_headers = []
    first_row_headers = []

    for th in first_row.find_all('th'):
        colspan = int(th.get('colspan', 1))  # Get the colspan value (default is 1 if not present)
        label = th.text.strip()

        first_row_headers.extend([label] * colspan)

    for idx, th in enumerate(second_row.find_all('th')):
        aria_label = th.text.strip()  

        broader_category = first_row_headers[idx]  
        # Combine 
        if broader_category:
            combined_header = f"{broader_category} - {aria_label}"
        else:
            combined_header = aria_label  # Use just the second row header if the first row is empty

        combined_headers.append(combined_header)

  
    flattened_headers = []
    for header in combined_headers:
        # Split headers into a more detailed format
        if " - " in header:
            category, sub_header = header.split(" - ", 1)
            flattened_headers.append(f"{category}-{sub_header}")
        else:
            flattened_headers.append(header)

    # Get player data as a list of dictionaries
    player_data = []
    tbody = table.find('tbody')
    rows = tbody.find_all('tr')

    # Extract data
    for row in rows:
        first_column = row.find('th').text.strip()  # The first column is usually rank
        columns = [col.text.strip() for col in row.find_all('td')]

        if len(columns) == 0:
            continue

        # Create a dictionary with the headers as keys and data as values
        player_info = {flattened_headers[0]: first_column}  # Rank
        for idx, column in enumerate(columns):
            player_info[flattened_headers[idx+1]] = column  # Use the flattened header for the key

        player_data.append(player_info)


    # Save into CSV
    df = pd.DataFrame(player_data)

    csv_filename = f"{filename_prefix}.csv"
    
    df.to_csv(csv_filename, index=False, encoding='utf-8')

    print(f"Data saved to {csv_filename}")

    driver.quit()

In [3]:
urls = {
    "standard_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_standard",  
        "table_id": "stats_standard"    
    },
    "goalkeeping_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/keepers/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_keeper",
        "table_id": "stats_keeper"
    },
    "shooting_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/shooting/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_shooting",
        "table_id": "stats_shooting"
    },
    "passing_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/passing/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_passing",
        "table_id": "stats_passing"
    },
    "pass_types_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/passing_types/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_passing_types",
        "table_id": "stats_passing_types"
    },
    "goal_and_shot_creation_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/gca/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_gca",
        "table_id": "stats_gca"
    },
    "defensive_actions_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/defense/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_defense",
        "table_id": "stats_defense"
    },
    "possession_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/possession/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_possession",
        "table_id": "stats_possession"
    },
    "playing_time_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/playingtime/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_playing_time",
        "table_id": "stats_playing_time"
    },
    "miscellaneous_stats": {
        "url": "https://fbref.com/en/comps/9/2023-2024/misc/2023-2024-Premier-League-Stats",
        "div_id": "div_stats_misc",
        "table_id": "stats_misc"
    }
}

In [4]:
if not os.path.exists('crawled'):
    os.makedirs('crawled')

# Fetch data
for stat_type, info in urls.items():
    fetch_and_save_stats(info['url'], f"./crawled/data_{stat_type}", info['div_id'], info['table_id'])

Data saved to ./crawled/data_standard_stats.csv
Data saved to ./crawled/data_goalkeeping_stats.csv
Data saved to ./crawled/data_shooting_stats.csv
Data saved to ./crawled/data_passing_stats.csv
Data saved to ./crawled/data_pass_types_stats.csv
Data saved to ./crawled/data_goal_and_shot_creation_stats.csv
Data saved to ./crawled/data_defensive_actions_stats.csv
Data saved to ./crawled/data_possession_stats.csv
Data saved to ./crawled/data_playing_time_stats.csv
Data saved to ./crawled/data_miscellaneous_stats.csv


In [5]:
files = [
    "./crawled/data_standard_stats.csv",
    "./crawled/data_goalkeeping_stats.csv",
    "./crawled/data_shooting_stats.csv",
    "./crawled/data_passing_stats.csv",
    "./crawled/data_pass_types_stats.csv",
    "./crawled/data_goal_and_shot_creation_stats.csv",
    "./crawled/data_defensive_actions_stats.csv",
    "./crawled/data_possession_stats.csv",
    "./crawled/data_playing_time_stats.csv",
    "./crawled/data_miscellaneous_stats.csv"
]

In [6]:
def read_csv(file):
    return pd.read_csv(file)

In [7]:
base_df = read_csv("./crawled/data_standard_stats.csv")

# Merge other dataframes onto the base dataframe
for file in files[1:]: 
    if os.path.exists(file):
        df = read_csv(file)
        base_df = pd.merge(base_df, df, on=['Player', 'Pos', 'Squad'], how='left', suffixes=('', '_y'))
        base_df = base_df.loc[:, ~base_df.columns.str.endswith('_y')]

In [8]:
# Replace missing statistics 
base_df = base_df.fillna('N/a')
base_df = base_df.drop(columns=['Rk', 'Matches'], errors='ignore')

# Ensure 'Playing Time-Min' is numeric 
base_df['Playing Time-Min'] = base_df['Playing Time-Min'].replace({',': ''}, regex=True)
base_df['Playing Time-Min'] = pd.to_numeric(base_df['Playing Time-Min'], errors='coerce')

# Players who have played more than 90 minutes 
filtered_df = base_df[base_df['Playing Time-Min'] > 90]

# Sort 
sorted_df = filtered_df.sort_values(by=['Player', 'Age'], ascending=[True, False])

In [9]:
# Print results
print(f"Data:")
print(sorted_df.head())  

Data:
                Player   Nation    Pos           Squad  Age  Born  \
117    Aaron Cresswell  eng ENG  DF,FW        West Ham   33  1989   
234       Aaron Hickey  sct SCO     DF       Brentford   21  2002   
436     Aaron Ramsdale  eng ENG     GK         Arsenal   25  1998   
437       Aaron Ramsey  eng ENG  MF,FW         Burnley   20  2003   
549  Aaron Wan-Bissaka  eng ENG     DF  Manchester Utd   25  1997   

     Playing Time-MP  Playing Time-Starts  Playing Time-Min  Playing Time-90s  \
117               11                    4               436               4.8   
234                9                    9               713               7.9   
436                6                    6               540               6.0   
437               14                    5               527               5.9   
549               22                   20              1780              19.8   

     ...  Performance-Crs  Performance-Int  Performance-TklW  \
117  ...               11   

In [10]:
# Save the results
sorted_df.to_csv("./crawled/merged_premier_league_stats.csv", index=False)
sorted_df.to_csv("./crawled/result.csv", index=False)